# Shell Element Responses (Plotly)

In [1]:
import openseespy.opensees as ops

import opstool as opst
import opstool.vis.plotly as opsvis


## Model and gravity load

In [2]:
opst.load_ops_examples("Shell3D")

ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)
_ = opst.pre.gen_grav_load(direction="Z", factor=-9810)

The original Tcl file comes from http://www.dinochen.com/, and the Python version is converted by opstool.tcl2py().


In [3]:
fig = opsvis.plot_model(show_nodal_loads=True, show_ele_loads=True)
fig.show()

# fig.show(renderer="jupyterlab")
# fig.show(renderer="notebook")
# fig.show(renderer="browser")
# fig.show()  # for auto
# fig.write_html("**.html")

In [4]:
fig.write_html("images/shell_plotly.html", full_html=False, include_plotlyjs="cdn")

## Gravity analysis

In [5]:
ops.constraints("Transformation")
ops.numberer("RCM")
ops.system("BandGeneral")
ops.test("NormDispIncr", 1.0e-6, 6, 2)
ops.algorithm("Linear")
ops.integrator("LoadControl", 0.1)
ops.analysis("Static")

In [6]:
ODB = opst.post.CreateODB(
    odb_tag=1,
    project_gauss_to_nodes="copy",  # project gauss point responses to nodes, optional ["copy", "average", "extrapolate"]
)
for _ in range(10):
    ops.analyze(1)
    ODB.fetch_response_step()
ODB.save_response()

OPSTOOL ::  All responses data with _odb_tag = 1 saved in .opstool.output/RespStepData-1.nc!

## Visualize the results

Nodal responses, ``project_gauss_to_nodes`` needs to be set to "copy", "average", or "extrapolate" when creating the ODB

In [7]:
opsvis.set_plot_props(cmap="rdylbu", point_size=2.0)

fig = opsvis.plot_unstruct_responses(
    odb_tag=1,
    slides=False,
    ele_type="Shell",
    resp_type="sectionForcesAtNodes",
    resp_dof="FXX",
    unit_symbol="kN",
    unit_factor=1,
    show_values=True,
)
fig.show()
# fig.write_html("**.html")

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...

In [8]:
fig.write_html("images/shell_resp_plotly.html", full_html=False, include_plotlyjs="cdn")

Display the responses at each element, all gauss points will be averaged to the element level.

In [9]:
fig = opsvis.plot_unstruct_responses(
    odb_tag=1,
    slides=True,
    ele_type="Shell",
    resp_type="sectionForces",  # element response, "AtGaussPoints", will be averaged to each element
    resp_dof="FXX",
    unit_symbol="kN",
    unit_factor=1,
    show_values=True,
)
fig.show()
fig.write_html("images/shell_resp_plotly2.html", full_html=False, include_plotlyjs="cdn")

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...

Fiber point stress can be plotted as well, but it requires a ``shell_fiber_loc`` to be assigned.

In [10]:
fig = opsvis.plot_unstruct_responses(
    odb_tag=1,
    slides=True,
    ele_type="Shell",
    resp_type="StressesAtNodes",  # nodal stress response, "AtNodes"
    resp_dof="sigma11",  # sigma11, sigma22, sigma12, sigma13, sigma23
    shell_fiber_loc="top",  # shell_fiber_loc can be "top", "bottom", or "mid" for shell elements, also int
    unit_symbol="kPa",
    unit_factor=1,  # unit_factor is used to convert the response value, if MPa, set it to 1e-3
    show_values=True,
)
fig.show()
fig.write_html("images/shell_resp_plotly3.html", full_html=False, include_plotlyjs="cdn")

OPSTOOL ::  Loading response data from .opstool.output/RespStepData-1.nc ...